In [ ]:
! pip install shaped

In [ ]:
API_KEY = "dQRowDdtmU4mCZdHyKALGGvYJxOo9Mw9Ye5bR544"

In [ ]:
import pandas as pd
import yaml
import json

In [ ]:
import shaped
import os
! shaped init --api-key {API_KEY} --env prod

Initializing with config: {'api_key': 'dQRowDdtmU4mCZdHyKALGGvYJxOo9Mw9Ye5bR544', 'env': 'prod'}


# Ranking

We can simply use the `rank` endpoint for your model to return a list of relevant items for a given user.

In [ ]:
# Define client
client = shaped.Client(api_key=API_KEY)

# Make a request to the shaped API
response = client.rank(
    model_name="h_and_m_product_recs_demo",
    user_id="a8f8851c9ca25414947073f42a377d522c89aeb3ce5769b2c2dbaa5bc7e7c2ee",
    return_metadata=True,
    config=shaped.InferenceConfig(
        exploration_factor=0.1,
        diversity_factor=0.1,
        retrieval_k=600,
        retrieval_k_override=shaped.RetrieverTopKOverride(
            knn=300,
            chronological=0,
            toplist=0,
            trending=300,
            random=0,
            cold_start=0,
        ),
        limit=5,
    ),
)

print(response)

ids=['751471043', '918292001', '919365008', '898694001', '933706001'] scores=[0.82786569, 0.82786569, 0.82786569, 0.82786569, 0.82786569] metadata=[{'item_id': '751471043', 'product_name': 'Pluto RW slacks (1)', 'product_type': 'Trousers', 'graphical_appearance': 'Check', 'category': 'Ladieswear', 'section': 'Womens Everyday Collection', 'department': 'Trouser', 'description': 'Ankle-length cigarette trousers in a stretch weave with a zip fly, concealed hook-and-eye fastening, and regular waist with concealed elastication. Side pockets, fake back pockets and tapered legs.', 'image': 'https://h-and-m-images.s3.us-east-2.amazonaws.com/075/0751471043.jpg', '_derived_chronological_rank': 27069, '_derived_trending_rank': 61, '_derived_popular_rank': 5, '_derived_interaction_count': 1710}, {'item_id': '918292001', 'product_name': 'STRONG HW seamless tights', 'product_type': 'Leggings/Tights', 'graphical_appearance': 'Melange', 'category': 'Sport', 'section': 'Ladies H&M Sport', 'department':

Similarly, we can use the `similar_items` endpoint to return a list of similar items for a candidate `item_id` :

In [ ]:
api_response = client.similar_items(
    model_name="h_and_m_product_recs_demo",
    item_id="150959011",
    return_metadata=True,
    config=shaped.InferenceConfig(
        exploration_factor=0.1,
        diversity_factor=0.1,
        retrieval_k=600,
        retrieval_k_override=shaped.RetrieverTopKOverride(
            knn=300,
            chronological=0,
            toplist=0,
            trending=300,
            random=0,
            cold_start=0,
        ),
        limit=3000,
    ),
)

print(api_response)

ids=['862104001', '760735001', '862106001', '825243003', '860241001', '862106003', '696125001', '904564001', '879827001', '862106004', '696125003', '893106001', '734592001', '866610001', '850040001', '680327001', '817820003', '720101001', '854532001', '866610002', '740519002', '609170009', '827184001', '740519006', '872705001', '872689001', '811891001', '863287003', '609170005', '836115001', '871424001', '609170010', '853992001', '871424002', '708132001', '719941015', '890677002', '865474001', '851318001', '890677001', '719941003', '862104009', '553271002', '719941001', '865477001', '736102009', '843458002', '224521007', '862104018', '862104002', '879827002', '862106006', '653716007', '816940001', '765734002', '736102011', '765734001', '736102005', '827256001', '553271004', '865912001', '924756001', '736102014', '533404010', '582258001', '825243001', '861737001', '853881001', '523404032', '817554001', '906575001', '890677004', '807362003', '843458005', '638899001', '810838010', '533406

In [ ]:
#hm_articles = pd.read_csv('h_and_m_articles.csv')
#hm_customers = pd.read_csv('h_and_m_customers.csv')
#hm_events = pd.read_csv('h_and_m_events.csv')

df_events = hm_events
df_customers = hm_customers
df_items = hm_articles

ids = api_response.ids

In [ ]:
import pandas as pd

# Assuming `hm_events`, `hm_customers`, and `ids` are already defined

# Step 1: Convert `ids` from strings to integers for efficient comparison with article_id (which is int)
ids = list(map(int, ids))  # Convert string IDs to integers

# Step 2: Create a dictionary mapping customer_id to age for faster lookups
customer_id_to_age = dict(zip(hm_customers['customer_id'], hm_customers['age']))

# Step 3: Create a dictionary of article_id -> customer_ids using groupby
# This step will precompute the mapping of each article_id to its list of customer_ids
article_to_customers = hm_events.groupby('article_id')['customer_id'].apply(list).to_dict()

# Step 4: Filter ids to those that actually appear in the `hm_events` article_id column
valid_ids = set(ids).intersection(article_to_customers.keys())

# Step 5: Iterate over valid item ids and calculate their average age
item_to_avg_age = {}

for item_id in valid_ids:
    # Get the customer IDs for this item_id
    customer_ids = article_to_customers[item_id]

    # Get the ages for these customer_ids using the precomputed customer_id_to_age
    ages = [customer_id_to_age[customer_id] for customer_id in customer_ids if customer_id in customer_id_to_age]

    if ages:
        avg_age = sum(ages) / len(ages)  # Calculate average age
        item_to_avg_age[str(item_id)] = avg_age

# Step 6: Define the age range
age_min, age_max = 20, 30  # Example range

# Step 7: Filter items based on the average age within the range
filtered_ids = [
    item_id
    for item_id, avg_age in item_to_avg_age.items()
    if age_min <= avg_age <= age_max
]

# Print only the first 3 item IDs from the filtered list
print(f"\nFirst 3 Filtered Item IDs (within the age range {age_min}-{age_max}):")
print(filtered_ids[:3])  # Slice the first 3 items

# Look up each item ID in the hm_articles dataset and print the corresponding "prod_name"
print("\nCorresponding Product Names:")
for item_id in filtered_ids[:3]:  # Only print for the first 3
    # Ensure that item_id is a string and look it up in the articles dataset
    product_info = hm_articles[hm_articles['article_id'].astype(str) == item_id]
    if not product_info.empty:
        print(f"Item ID: {item_id}, Product Name: {product_info['prod_name'].iloc[0]}")
    else:
        print(f"Item ID: {item_id} not found in the articles dataset.")



First 3 Filtered Item IDs (within the age range 27-27):
['868393001', '811385001', '843465023']

Corresponding Product Names:
Item ID: 868393001, Product Name: Lilja trouser
Item ID: 811385001, Product Name: PQ PRESLEY TRS
Item ID: 843465023, Product Name: Jack Skinny Trousers PID
